# Pyddm
## &copy;  [Omkar Mehta](omehta2@illinois.edu) ##
### Industrial and Enterprise Systems Engineering, The Grainger College of Engineering,  UIUC ###

<hr style="border:2px solid blue"> </hr>


PyDDM can simulate models and generate artificial data, or it can fit them to data. Below are high-level overviews for how to accomplish each.

To simulate models and generate artificial data:

1. Optionally, define unique components of your model. Models are modular, and allow specifying a dynamic drift rate, noise level, diffusion bounds, starting position of the integrator, or post-simulation modifications to the RT histogram. Many common models for these are included by default, but for advance functionality you may need to subclass [Drift](https://pyddm.readthedocs.io/en/stable/apidoc/dependences.html#ddm.models.drift.Drift), [Noise](https://pyddm.readthedocs.io/en/stable/apidoc/dependences.html#ddm.models.noise.Noise), [Bound](https://pyddm.readthedocs.io/en/stable/apidoc/dependences.html#ddm.models.bound.Bound), [InitialCondition](https://pyddm.readthedocs.io/en/stable/apidoc/dependences.html#ddm.models.ic.InitialCondition), or [Overlay](https://pyddm.readthedocs.io/en/stable/apidoc/dependences.html#ddm.models.overlay.Overlay). These model components may depend on “conditions”, i.e. prespecified values associated with the behavioral task which change from trial to trial (e.g. stimulus coherence), or “parameters”, i.e. values which apply to all trials and should be fit to the subject.
2. Define a model. Models are represented by creating an instance of the [Model](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.model.Model) class, and specifying the model components to use for it. These model component can [either be the model components included in PyDDM](https://pyddm.readthedocs.io/en/stable/apidoc/dependences.html) or ones you created in step 1. Values must be specified for all parameters required by model components.
3. Simulate the model using the [Model.solve()](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.model.Model.solve) method to generate a [Solution](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.solution.Solution) object. If you have multiple conditions, you must run [Model.solve()](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.model.Model.solve) separately for each set of conditions and generate separate [Solution](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.solution.Solution) objects.
4. Run the [Solution.resample()](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.solution.Solution.resample) method of the [Solution](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.solution.Solution) object to generate a [Sample](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.sample.Sample). If you have multiple [Solution](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.solution.Solution) objects (for multiple task conditions), you will need to generate multiple [Sample](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.sample.Sample) objects as well. These can be added together with the “+” operator to form one single [Sample](https://pyddm.readthedocs.io/en/stable/apidoc/model.html#ddm.sample.Sample) object.


In [ ]:
!pip install scipy
!pip install numpy
!pip install pyddm
!pip install matplotlib
!pip install pandas
!pip install paranoid-scientist
!pip install pathos
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 73 kB 842 kB/s 
     |████████████████████████████████| 10.3 MB 4.6 MB/s 
     |████████████████████████████████| 67 kB 934 kB/s 
     |████████████████████████████████| 1.2 MB 4.8 MB/s 
     |████████████████████████████████| 3.0 MB 492 kB/s 
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defa

In [5]:
# libraries
from ddm import Model
from ddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision
from ddm.functions import fit_adjust_model, display_model

model = Model(name='Simple model',
              drift=DriftConstant(drift=2.0),
              noise=NoiseConstant(noise=1.5),
              bound=BoundConstant(B=1.1),
              overlay=OverlayNonDecision(nondectime=.1),
              dx=.001, dt=.01, T_dur=2)


In [16]:
print(f'Number of timesteps is {2/0.01+1}')  # considers initial state
model.t_domain()
print(f"Shape of t_domain() is {model.t_domain().shape}")
# print(f'Number of x_steps is {1.1/0.001+1}') since B=1.1
print(f'Number of x_steps is {2*1.1/0.001+1}')

Number of timesteps is 201.0
Shape of t_domain() is (201,)
Number of x_steps is 2201.0


In [4]:
display_model(model)

Model Simple model information:
Drift component DriftConstant:
    constant
    Fixed parameters:
    - drift: 2.200000
Noise component NoiseConstant:
    constant
    Fixed parameters:
    - noise: 1.500000
Bound component BoundConstant:
    constant
    Fixed parameters:
    - B: 1.100000
IC component ICPointSourceCenter:
    point_source_center
    (No parameters)
Overlay component OverlayNonDecision:
    Add a non-decision by shifting the histogram
    Fixed parameters:
    - nondectime: 0.100000



In [19]:
# simulate_trial() simulates decision variable for a single trial. Uses Runge-Kutta by default.
model.simulate_trial()

array([-1.21236354e-13, -1.21236354e-13, -1.21236354e-13, -1.21236354e-13,
       -1.21236354e-13, -1.21236354e-13, -1.21236354e-13, -1.21236354e-13,
       -1.21236354e-13, -1.21236354e-13, -1.21236354e-13,  1.31238761e-01,
        3.84175819e-01,  6.39265892e-02,  2.83958396e-01,  1.77549788e-01,
        4.93038456e-01,  7.02956235e-01,  6.47074754e-01,  1.04885487e+00,
        1.23097666e+00])

In [ ]:
# simulated_solution() simulate individual trials (simulate_trial) 1000 times by default to yield a histogram.
# model.simulated_solution() 

In [6]:
# solve() solves the model using an analytical solution and returns a Solution object describing the joing PDF distrobution of reaction times
sol = model.solve(return_evolution=True)

In [7]:
# returns the pdf of correct reaction times
len(sol.pdf_corr())

201

In [21]:
print(sol.corr)

print('\n')

print(sol.pdf_corr())

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 7.88741941e-05
 4.07558114e-04 1.15395962e-03 2.38553265e-03 4.04539430e-03
 5.99153633e-03 8.05449549e-03 1.00817200e-02 1.19595367e-02
 1.36171456e-02 1.50206115e-02 1.61632978e-02 1.70564869e-02
 1.77217854e-02 1.81856426e-02 1.84757301e-02 1.86187457e-02
 1.86392178e-02 1.85589684e-02 1.83969821e-02 1.81695104e-02
 1.78902979e-02 1.75708619e-02 1.72207811e-02 1.68479734e-02
 1.64589483e-02 1.60590309e-02 1.56525566e-02 1.52430372e-02
 1.48333016e-02 1.44256151e-02 1.40217778e-02 1.36232075e-02
 1.32310083e-02 1.28460275e-02 1.24689022e-02 1.21000983e-02
 1.17399423e-02 1.13886478e-02 1.10463369e-02 1.07130582e-02
 1.03888013e-02 1.00735085e-02 9.76708522e-03 9.46940753e-03
 9.18032894e-03 8.89968570e-03 8.62730123e-03 8.36298964e-03
 8.10655864e-03 7.85781189e-03 7.61655086e-03 7.38257639e-03
 7.15568989e-03 6.935694

In [11]:
# stores the pdf evolution of the reaction times. Returns x_domain by t_domain numpy error
pdf_evol = sol.pdf_evolution()

In [13]:
# print(f'Number of x_steps is {1.1/0.001+1}') since B=1.1
pdf_evol.shape

(2201, 201)

## Plot PDF evolution using Plotly


In [ ]:
import plotly.graph_objects as go

z=pdf_evol
sh_0, sh_1 = z.shape
x, y = np.linspace(0, 2.2, sh_0), np.linspace(0, 2, sh_1)
fig = go.Figure(data=[go.Surface(z=z)])

fig.update_layout(title='PDF Evolution', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()


In [21]:
samp=sol.resample(k = 10)

In [22]:
samp_df = samp.to_pandas_dataframe()

In [23]:
samp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   correct  10 non-null     int64  
 1   RT       10 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 288.0 bytes


In [24]:
samp_df

,correct,RT
0,1,1.070170
1,1,0.193883
2,1,0.359876
3,1,0.590755
4,1,0.391765
5,1,0.568277
6,1,1.494951
7,1,0.259402
8,1,0.409133
9,0,0.373702


In [26]:
samp.items(correct=True)

In [28]:
samp.condition_names()

[]

In [29]:
samp.t_domain()

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09,
       1.1 , 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 ,
       1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31,
       1.32, 1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42,
       1.43, 1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.

In [30]:
samp.pdf_corr()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,
       10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,
       10.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., 10.,  0., 10.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [32]:
samp.corr

array([1.07017047, 0.19388311, 0.35987629, 0.59075526, 0.39176538,
       0.56827721, 1.49495124, 0.25940165, 0.40913304])